In [1]:
import librosa
import librosa.display
from librosa.core import load
import numpy as np
import pandas as pd
import glob
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
import time

### Load Data

In [2]:
VERSION = 4 
df_joint_train = pd.read_csv(f'./features/cache_all_features_train_V{VERSION}.csv')
feature_column_names = [i for i in df_joint_train.columns \
                        if i not in ['file_path','renamed_file_path','split','sentiment_value','emotional_category']]

df_joint_test = pd.read_csv(f'./features/cache_all_features_test_V{VERSION}.csv')
print("df_joint_train shape: ", df_joint_train.shape)
print("df_joint_test shape: ", df_joint_test.shape)

feat_rank_emo = pd.read_csv('./features/relationship_stats_test_result_emotion_all_V4.csv')
feat_rank_sti = pd.read_csv('./features/relationship_stats_test_result_sentiment_all_V4.csv')
feat_rank_emo.correlation_coeff = abs(feat_rank_emo.correlation_coeff)
feat_rank_sti.correlation_coeff = abs(feat_rank_sti.correlation_coeff)
feature_types = ['mfcc','chroma','mel','zcr','rms','Intensity','Pitch','GNE','HNR','Spectrum','Formant']

df_joint_train shape:  (10981, 1550)
df_joint_test shape:  (1180, 1550)


In [3]:
X_train = df_joint_train[feature_column_names]
y_train_s = df_joint_train['sentiment_value']
y_train_e = df_joint_train['emotional_category']

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_e_num = label_encoder.fit_transform(y_train_e)

X_test = df_joint_test[feature_column_names]
y_test_s = df_joint_test['sentiment_value']
y_test_e = df_joint_test['emotional_category']

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_test_e_num = label_encoder.fit_transform(y_test_e)

### Check feature ranking

In [4]:
feat_rank_emo.describe()

,p_value,is_significant,f_score,correlation_coeff,mutual_info_score
count,1546.00000,1546.000000,1546.000000,1546.000000,1546.000000
mean,0.00013,0.999353,79.626271,0.113689,0.145012
std,0.00216,0.025433,83.848850,0.064382,0.064789
min,0.00000,0.000000,1.989100,0.000300,0.004300
25%,0.00000,1.000000,28.025975,0.058900,0.094050
50%,0.00000,1.000000,60.002250,0.113300,0.151750
75%,0.00000,1.000000,105.034900,0.160200,0.204300
max,0.05270,1.000000,876.182000,0.400300,0.307400


In [5]:
features_score_above_p50 = feat_rank_emo[(feat_rank_emo.f_score>60) &
(feat_rank_emo.mutual_info_score>0.15)&
(feat_rank_emo.correlation_coeff>0.11)]['feature'].values

print("features with score above p50 from Emotion label: ", len(features_score_above_p50))
print(features_score_above_p50)

features with score above p50 from Emotion label:  466
['mfcc_p90_1' 'Intensity_max_intensity' 'Intensity_q3_intensity'
 'mfcc_median_1' 'mfcc_mean_1' 'Spectrum_band_energy_difference'
 'Spectrum_band_density_difference' 'rms_std' 'Pitch_mean_pitch'
 'mfcc_p10_3' 'mfcc_std_1' 'rms_p90' 'Formant_f1_median'
 'Pitch_median_intensity' 'mfcc_mean_3' 'Pitch_q1_pitch' 'rms_mean'
 'mfcc_std_19' 'mfcc_std_21' 'rms_median' 'mfcc_std_20' 'Pitch_min_pitch'
 'mfcc_std_24' 'Intensity_median_intensity' 'mfcc_std_22'
 'Spectrum_center_of_gravity_spectrum' 'zcr_median' 'GNE_sum_gne'
 'GNE_mean_gne' 'mel32_std_22' 'mel64_std_43' 'mel32_std_23'
 'mel32_std_21' 'mel32_mean_22' 'mel32_mean_21' 'mel32_std_26'
 'mel32_mean_26' 'mel64_mean_42' 'mel32_mean_23' 'mel64_std_42'
 'mel64_mean_43' 'mel32_std_25' 'mel32_mean_20' 'mel32_mean_24'
 'mel128_mean_84' 'mel32_std_24' 'mel64_mean_51' 'mel128_std_85'
 'mel32_std_20' 'mel32_mean_25' 'mel128_std_84' 'mel64_std_46'
 'mel64_mean_46' 'mel64_std_51' 'mel64_std_41' 

In [6]:
feat_rank_emo[feat_rank_emo.feature.str.contains('Pitch')]

,feature,p_value,is_significant,f_score,correlation_coeff,mutual_info_score
2,Pitch_pitch_slope_without_octave_jumps,0.0,1,756.2722,0.0610,0.1888
9,Pitch_q3_pitch,0.0,1,486.9693,0.0804,0.2173
10,Pitch_mean_pitch,0.0,1,474.9486,0.1277,0.2180
12,Pitch_mean_absolute_pitch_slope,0.0,1,458.2618,0.0704,0.1425
18,Pitch_median_intensity,0.0,1,392.1258,0.1407,0.1977
20,Pitch_q1_pitch,0.0,1,369.6490,0.1797,0.2066
31,Pitch_stddev_pitch,0.0,1,316.3860,0.0627,0.1463
42,Pitch_min_pitch,0.0,1,288.2170,0.1438,0.1891
64,Pitch_max_pitch,0.0,1,237.3622,0.0223,0.1799
558,Pitch_voiced_fraction,0.0,1,79.7698,0.0842,0.0593


In [7]:
features_score_above_p75 = feat_rank_sti[(feat_rank_sti.f_score>52) & 
(feat_rank_sti.correlation_coeff>0.058) & 
(feat_rank_sti.mutual_info_score>0.05)]['feature'].values 

print("features with score above p75 from Sentiment label: ", len(features_score_above_p75))
print(features_score_above_p75)

features with score above p75 from Sentiment label:  68
['Pitch_pitch_slope_without_octave_jumps' 'Pitch_stddev_pitch'
 'Pitch_mean_absolute_pitch_slope' 'Pitch_q3_pitch' 'mfcc_median_33'
 'Pitch_mean_pitch' 'mfcc_mean_33' 'mfcc_median_34' 'mfcc_median_32'
 'Pitch_max_pitch' 'Pitch_median_intensity' 'mfcc_std_21' 'mfcc_mean_32'
 'mfcc_mean_34' 'mfcc_std_22' 'mfcc_std_27' 'mfcc_std_24' 'mfcc_std_23'
 'mfcc_std_20' 'mfcc_p90_22' 'mfcc_std_17' 'mfcc_std_19' 'mfcc_p90_33'
 'Intensity_q3_intensity' 'mfcc_p90_25' 'mfcc_std_18' 'mfcc_std_1'
 'Pitch_min_pitch' 'mel32_std_22' 'mel32_std_23' 'Local Jitter'
 'mel64_std_43' 'mel32_mean_22' 'mel32_mean_23' 'mel64_mean_46'
 'mel64_p10_3' 'mel32_mean_24' 'mel32_std_12' 'mel64_mean_43'
 'mel64_std_42' 'mel64_std_44' 'mel64_mean_42' 'mel32_std_9'
 'mel128_mean_91' 'mel64_mean_44' 'mel32_p90_24' 'mel128_std_84'
 'mel128_mean_84' 'mel64_mean_45' 'mel64_mean_48' 'mel64_std_45'
 'mel128_mean_99' 'mel32_std_11' 'mel32_p90_23' 'mel64_mean_47'
 'mel32_p90_22'

### import Model

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, precision_recall_fscore_support

dt_clf= DecisionTreeClassifier()
rf_clf = RandomForestClassifier()

def try_clf_with_feature_selected(clf_model, feature_names,y_train,y_test):
    X_train = df_joint_train[feature_names]
    X_test = df_joint_test[feature_names]
    predictions = clf_model.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_test, predictions ))
    print(confusion_matrix(y_test, predictions))

In [9]:
try_clf_with_feature_selected(dt_clf,features_score_above_p75,y_train_s,y_test_s)

              precision    recall  f1-score   support

          -1       0.76      0.81      0.78       748
           0       0.53      0.49      0.51       183
           1       0.58      0.49      0.53       249

    accuracy                           0.69      1180
   macro avg       0.62      0.60      0.61      1180
weighted avg       0.69      0.69      0.69      1180

[[607  59  82]
 [ 86  90   7]
 [107  20 122]]


## Forward Selection by category
- select certain category features
- remove low related features (manual study results in selected_<group>) 
- apply auto feature selection to determin how many feature chosen
- OPTIONAL experiment how much performance drop from full -> selected -> less -> 2PCs?

- Alternative is too do prosody features together AUTO

### Prodosy Features List
Total 56   
['Intensity_min_intensity', 'Intensity_relative_min_intensity_time', 'Intensity_max_intensity', 'Intensity_relative_max_intensity_time', 'Intensity_mean_intensity', 'Intensity_stddev_intensity', 'Intensity_q1_intensity', 'Intensity_median_intensity', 'Intensity_q3_intensity', 'Pitch_voiced_fraction', 'Pitch_min_pitch', 'Pitch_relative_min_pitch_time', 'Pitch_max_pitch', 'Pitch_relative_max_pitch_time', 'Pitch_mean_pitch', 'Pitch_stddev_pitch', 'Pitch_q1_pitch', 'Pitch_median_intensity', 'Pitch_q3_pitch', 'Pitch_mean_absolute_pitch_slope', 'Pitch_pitch_slope_without_octave_jumps', 'GNE_max_gne', 'GNE_mean_gne', 'GNE_stddev_gne', 'GNE_sum_gne', 'HNR_min_hnr', 'HNR_relative_min_hnr_time', 'HNR_max_hnr', 'HNR_relative_max_hnr_time', 'HNR_mean_hnr', 'HNR_stddev_hnr', 'Spectrum_band_energy', 'Spectrum_band_density', 'Spectrum_band_energy_difference', 'Spectrum_band_density_difference', 'Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum', 'Spectrum_skewness_spectrum', 'Spectrum_kurtosis_spectrum', 'Spectrum_central_moment_spectrum', 'Formant_f1_mean', 'Formant_f2_mean', 'Formant_f3_mean', 'Formant_f4_mean', 'Formant_f1_median', 'Formant_f2_median', 'Formant_f3_median', 'Formant_f4_median', 'Formant_formant_dispersion', 'Formant_average_formant', 'Formant_mff', 'Formant_fitch_vtl', 'Formant_delta_f', 'Formant_vtl_delta_f', 'Local Jitter', 'Local Shimmer']

In [10]:
feature_names_prosody = ['Intensity_min_intensity', 'Intensity_relative_min_intensity_time', 'Intensity_max_intensity', 'Intensity_relative_max_intensity_time', 'Intensity_mean_intensity', 'Intensity_stddev_intensity', 'Intensity_q1_intensity', 'Intensity_median_intensity', 'Intensity_q3_intensity', 
                         'Pitch_voiced_fraction', 'Pitch_min_pitch', 'Pitch_relative_min_pitch_time', 'Pitch_max_pitch', 'Pitch_relative_max_pitch_time', 'Pitch_mean_pitch', 'Pitch_stddev_pitch', 'Pitch_q1_pitch', 'Pitch_median_intensity', 'Pitch_q3_pitch', 'Pitch_mean_absolute_pitch_slope', 'Pitch_pitch_slope_without_octave_jumps', 
                         'GNE_max_gne', 'GNE_mean_gne', 'GNE_stddev_gne', 'GNE_sum_gne', 'HNR_min_hnr', 'HNR_relative_min_hnr_time', 'HNR_max_hnr', 'HNR_relative_max_hnr_time', 'HNR_mean_hnr', 'HNR_stddev_hnr', 
                         'Spectrum_band_energy', 'Spectrum_band_density', 'Spectrum_band_energy_difference', 'Spectrum_band_density_difference', 'Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum', 'Spectrum_skewness_spectrum', 'Spectrum_kurtosis_spectrum', 'Spectrum_central_moment_spectrum', 'Formant_f1_mean', 'Formant_f2_mean', 'Formant_f3_mean', 'Formant_f4_mean', 'Formant_f1_median', 'Formant_f2_median', 'Formant_f3_median', 'Formant_f4_median', 'Formant_formant_dispersion', 'Formant_average_formant', 'Formant_mff', 'Formant_fitch_vtl', 'Formant_delta_f', 'Formant_vtl_delta_f', 
                         'Local Jitter', 'Local Shimmer']
print(len(feature_names_prosody))
# feature_names_prosody_selected = [ name for name in feature_names_prosody if name in features_score_above_p50 ]
# selected_spect = ['Spectrum_band_energy_difference','Spectrum_band_density_difference','Spectrum_center_of_gravity_spectrum','Spectrum_skewness_spectrum','Spectrum_kurtosis_spectrum', 'Spectrum_stddev_spectrum','Spectrum_band_density', 'Spectrum_band_energy']
# selected_formant = ['Formant_f1_mean','Formant_f1_median','Formant_f3_mean','Formant_fitch_vtl','Formant_mff','Formant_formant_dispersion']
# selected_GNE = ['GNE_max_gne', 'GNE_stddev_gne', 'GNE_mean_gne', 'GNE_sum_gne']
# selected_pitch = ['Pitch_pitch_slope_without_octave_jumps', 'Pitch_q3_pitch','Pitch_stddev_pitch', 'Pitch_mean_absolute_pitch_slope','Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Pitch_min_pitch']
# selected_intensity = ['Intensity_max_intensity', 'Intensity_q3_intensity','Intensity_median_intensity', 'Intensity_mean_intensity', 'Intensity_stddev_intensity','Intensity_relative_max_intensity_time']
# selected_HNR = ['HNR_stddev_hnr', 'HNR_mean_hnr','HNR_relative_min_hnr_time','HNR_max_hnr']
# feature_names_prosody_selected = selected_spect + selected_formant + selected_HNR + selected_intensity + selected_pitch + ['Local Jitter','Local Shimmer']
# print(len(feature_names_prosody_selected))

56


In [11]:
from sklearn.feature_selection import SequentialFeatureSelector

rf_est = RandomForestClassifier(n_estimators=25, random_state=42)
sfs = SequentialFeatureSelector(estimator=rf_est, 
                                n_features_to_select='auto',  # Select one feature at a time
                                tol = 0.001, # minimum amount incremented between two consecutive additions,
                                direction='forward',  # Forward selection
                                scoring='balanced_accuracy',  # Use balabced accuracy as the evaluation metric - average of recall
                                cv=5)  # k-fold cross-validation

In [12]:
def forward_select(group_name):
    start = time.time()
    features = [ name for name in feature_names_prosody if group_name in name ]
    X_train = df_joint_train[features]
    X_test  = df_joint_test[features]
    sfs.fit(X_train, y_train_s)

    selected_feature_names = [features[i] for i, selected in enumerate(sfs.get_support()) if selected]
    print(f"{len(selected_feature_names)} of {len(features)} selected. \nFeature names:", selected_feature_names)
    X_train_selected = sfs.transform(X_train)
    X_test_selected = sfs.transform(X_test)
    rf_est.fit(X_train_selected, y_train_s)
    accuracy = rf_est.score(X_test_selected, y_test_s)
    print("Accuracy with selected features:", accuracy)
    print(f"time taken: {round(time.time()-start)} seconds.")
    return selected_feature_names

# forward_select('Intensity')

In [13]:
selected_prosody = []
for group in ['Pitch','Intensity','HNR','Spectrum','Formant']:
    selected_prosody.extend(forward_select(group))

5 of 12 selected. 
Feature names: ['Pitch_voiced_fraction', 'Pitch_min_pitch', 'Pitch_q1_pitch', 'Pitch_q3_pitch', 'Pitch_pitch_slope_without_octave_jumps']
Accuracy with selected features: 0.7220338983050848
time taken: 72 seconds.
5 of 9 selected. 
Feature names: ['Intensity_max_intensity', 'Intensity_relative_max_intensity_time', 'Intensity_stddev_intensity', 'Intensity_q1_intensity', 'Intensity_q3_intensity']
Accuracy with selected features: 0.6940677966101695
time taken: 49 seconds.
2 of 6 selected. 
Feature names: ['HNR_relative_min_hnr_time', 'HNR_mean_hnr']
Accuracy with selected features: 0.5915254237288136
time taken: 17 seconds.
4 of 9 selected. 
Feature names: ['Spectrum_band_energy', 'Spectrum_band_energy_difference', 'Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum']
Accuracy with selected features: 0.6516949152542373
time taken: 40 seconds.
1 of 14 selected. 
Feature names: ['Formant_f1_mean']
Accuracy with selected features: 0.49322033898305084
time take

In [14]:
len(selected_prosody), selected_prosody

(17,
 ['Pitch_voiced_fraction',
  'Pitch_min_pitch',
  'Pitch_q1_pitch',
  'Pitch_q3_pitch',
  'Pitch_pitch_slope_without_octave_jumps',
  'Intensity_max_intensity',
  'Intensity_relative_max_intensity_time',
  'Intensity_stddev_intensity',
  'Intensity_q1_intensity',
  'Intensity_q3_intensity',
  'HNR_relative_min_hnr_time',
  'HNR_mean_hnr',
  'Spectrum_band_energy',
  'Spectrum_band_energy_difference',
  'Spectrum_center_of_gravity_spectrum',
  'Spectrum_stddev_spectrum',
  'Formant_f1_mean'])

In [15]:
try_clf_with_feature_selected(dt_clf,selected_prosody,y_train_s,y_test_s)

              precision    recall  f1-score   support

          -1       0.80      0.78      0.79       748
           0       0.55      0.61      0.58       183
           1       0.57      0.56      0.56       249

    accuracy                           0.71      1180
   macro avg       0.64      0.65      0.64      1180
weighted avg       0.71      0.71      0.71      1180

[[584  73  91]
 [ 55 112  16]
 [ 91  19 139]]


## now try backwar/d

In [17]:
def backward_select(group_name):
    start = time.time()
    features = [ name for name in feature_names_prosody if group_name in name ]
    X_train = df_joint_train[features]
    X_test  = df_joint_test[features]
    sfs_bk = SequentialFeatureSelector(estimator=rf_est, 
                                n_features_to_select='auto',  # Select one feature at a time
                                direction='backward',  # Forward selection
                                scoring='balanced_accuracy',  # Use balabced accuracy as the evaluation metric - average of recall
                                cv=5)  # k-fold cross-validation
    sfs_bk.fit(X_train, y_train_s)

    selected_feature_names = [features[i] for i, selected in enumerate(sfs_bk.get_support()) if selected]
    print(f"{len(selected_feature_names)} of {len(features)} selected. \nFeature names:", selected_feature_names)
    X_train_selected = sfs_bk.transform(X_train)
    X_test_selected = sfs_bk.transform(X_test)
    rf_est.fit(X_train_selected, y_train_s)
    accuracy = rf_est.score(X_test_selected, y_test_s)
    print("Accuracy with selected features:", accuracy)
    print(f"time taken: {round(time.time()-start)} seconds.")
    return selected_feature_names

# backward_select('Pitch')

In [18]:
selected_prosody_bk = []
for group in ['Pitch','Intensity','HNR','Spectrum','Formant']:
    selected_prosody_bk.extend(backward_select(group))
print(len(selected_prosody_bk), selected_prosody_bk)
try_clf_with_feature_selected(dt_clf,selected_prosody_bk,y_train_s,y_test_s)

6 of 12 selected. 
Feature names: ['Pitch_voiced_fraction', 'Pitch_min_pitch', 'Pitch_stddev_pitch', 'Pitch_q1_pitch', 'Pitch_q3_pitch', 'Pitch_pitch_slope_without_octave_jumps']
Accuracy with selected features: 0.7305084745762712
time taken: 113 seconds.
5 of 9 selected. 
Feature names: ['Intensity_relative_min_intensity_time', 'Intensity_max_intensity', 'Intensity_relative_max_intensity_time', 'Intensity_stddev_intensity', 'Intensity_median_intensity']
Accuracy with selected features: 0.6838983050847458
time taken: 49 seconds.
3 of 6 selected. 
Feature names: ['HNR_relative_min_hnr_time', 'HNR_max_hnr', 'HNR_mean_hnr']
Accuracy with selected features: 0.614406779661017
time taken: 25 seconds.
5 of 9 selected. 
Feature names: ['Spectrum_band_energy', 'Spectrum_band_energy_difference', 'Spectrum_band_density_difference', 'Spectrum_center_of_gravity_spectrum', 'Spectrum_skewness_spectrum']
Accuracy with selected features: 0.688135593220339
time taken: 48 seconds.
7 of 14 selected. 
Feat

## simple filter based on ranking -result 

In [32]:
selected_pitch = ['Pitch_pitch_slope_without_octave_jumps', 'Pitch_q3_pitch','Pitch_stddev_pitch', 
                  'Pitch_mean_absolute_pitch_slope','Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Pitch_min_pitch']
selected_intensity = ['Intensity_max_intensity', 'Intensity_q3_intensity','Intensity_median_intensity', 
                      'Intensity_mean_intensity', 'Intensity_stddev_intensity','Intensity_relative_max_intensity_time']
selected_spect = ['Spectrum_band_energy_difference','Spectrum_band_density_difference','Spectrum_center_of_gravity_spectrum',
                  'Spectrum_skewness_spectrum','Spectrum_kurtosis_spectrum', 'Spectrum_stddev_spectrum','Spectrum_band_density', 'Spectrum_band_energy']
selected_formant = ['Formant_f1_mean','Formant_f1_median','Formant_f3_mean','Formant_fitch_vtl','Formant_mff','Formant_formant_dispersion']
selected_HNR = ['HNR_stddev_hnr', 'HNR_mean_hnr','HNR_relative_min_hnr_time','HNR_max_hnr']
feature_names_prosody_filtered= selected_spect + selected_formant + selected_HNR + selected_intensity + selected_pitch + ['Local Jitter','Local Shimmer']
len(feature_names_prosody_filtered)

34

## try backward selection 

In [19]:
feature_names_prosody = ['Intensity_min_intensity', 'Intensity_relative_min_intensity_time', 'Intensity_max_intensity',
                         'Intensity_relative_max_intensity_time', 'Intensity_mean_intensity', 'Intensity_stddev_intensity', 
                         'Intensity_q1_intensity', 'Intensity_median_intensity', 'Intensity_q3_intensity', 
                         'Pitch_voiced_fraction', 'Pitch_min_pitch', 'Pitch_relative_min_pitch_time', 'Pitch_max_pitch', 
                         'Pitch_relative_max_pitch_time', 'Pitch_mean_pitch', 'Pitch_stddev_pitch', 'Pitch_q1_pitch', 
                         'Pitch_median_intensity', 'Pitch_q3_pitch', 'Pitch_mean_absolute_pitch_slope', 'Pitch_pitch_slope_without_octave_jumps',
                         'HNR_min_hnr', 'HNR_relative_min_hnr_time', 'HNR_max_hnr', 'HNR_relative_max_hnr_time', 'HNR_mean_hnr', 'HNR_stddev_hnr', 
                         'Spectrum_band_energy', 'Spectrum_band_density', 'Spectrum_band_energy_difference', 'Spectrum_band_density_difference', 'Spectrum_center_of_gravity_spectrum', 
                         'Spectrum_stddev_spectrum', 'Spectrum_skewness_spectrum', 'Spectrum_kurtosis_spectrum', 'Spectrum_central_moment_spectrum', 
                         'Formant_f1_mean', 'Formant_f2_mean', 'Formant_f3_mean', 'Formant_f4_mean', 'Formant_f1_median', 
                         'Formant_f2_median', 'Formant_f3_median', 'Formant_f4_median', 'Formant_formant_dispersion', 'Formant_average_formant', 
                         'Formant_mff', 'Formant_fitch_vtl', 'Formant_delta_f', 'Formant_vtl_delta_f', 
                         'Local Jitter', 'Local Shimmer']

In [21]:
start = time.time()
X_train = df_joint_train[feature_names_prosody]
X_test  = df_joint_test[feature_names_prosody]
rf_est = RandomForestClassifier(n_estimators=25)
sfs_bk = SequentialFeatureSelector(estimator=rf_est, 
                            n_features_to_select='auto',  # Select one feature at a time
                            direction='backward',  # Forward selection
                            scoring='balanced_accuracy',  # Use balabced accuracy as the evaluation metric - average of recall
                            cv=3)  # k-fold cross-validation
sfs_bk.fit(X_train, y_train_s)

selected_feature_names = [feature_names_prosody[i] for i, selected in enumerate(sfs_bk.get_support()) if selected]
print(f"{len(selected_feature_names)} of {len(feature_names_prosody)} selected. \nFeature names:", selected_feature_names)
X_train_selected = sfs_bk.transform(X_train)
X_test_selected = sfs_bk.transform(X_test)
rf_est.fit(X_train_selected, y_train_s)
accuracy = rf_est.score(X_test_selected, y_test_s)
print("Accuracy with selected features:", accuracy)
print(f"time taken: {round(time.time()-start)} seconds.")
print("selected: ", selected_feature_names)

26 of 52 selected. 
Feature names: ['Intensity_relative_min_intensity_time', 'Intensity_max_intensity', 'Intensity_q3_intensity', 'Pitch_voiced_fraction', 'Pitch_relative_max_pitch_time', 'Pitch_mean_pitch', 'Pitch_stddev_pitch', 'Pitch_median_intensity', 'Pitch_q3_pitch', 'Pitch_mean_absolute_pitch_slope', 'Pitch_pitch_slope_without_octave_jumps', 'HNR_relative_max_hnr_time', 'Spectrum_band_energy', 'Spectrum_band_density_difference', 'Spectrum_skewness_spectrum', 'Spectrum_central_moment_spectrum', 'Formant_f4_mean', 'Formant_f1_median', 'Formant_f4_median', 'Formant_formant_dispersion', 'Formant_average_formant', 'Formant_mff', 'Formant_fitch_vtl', 'Formant_delta_f', 'Local Jitter', 'Local Shimmer']
Accuracy with selected features: 0.7661016949152543
time taken: 2076 seconds.
selected:  ['Intensity_relative_min_intensity_time', 'Intensity_max_intensity', 'Intensity_q3_intensity', 'Pitch_voiced_fraction', 'Pitch_relative_max_pitch_time', 'Pitch_mean_pitch', 'Pitch_stddev_pitch', 'Pit

In [26]:
try_clf_with_feature_selected(rf_clf,feature_names_prosody,y_train_s,y_test_s)
try_clf_with_feature_selected(rf_clf,selected_feature_names,y_train_s,y_test_s)

              precision    recall  f1-score   support

          -1       0.76      0.96      0.85       748
           0       0.77      0.48      0.59       183
           1       0.91      0.45      0.60       249

    accuracy                           0.77      1180
   macro avg       0.81      0.63      0.68      1180
weighted avg       0.79      0.77      0.75      1180

[[715  23  10]
 [ 94  88   1]
 [134   4 111]]
              precision    recall  f1-score   support

          -1       0.75      0.96      0.84       748
           0       0.79      0.44      0.56       183
           1       0.89      0.45      0.60       249

    accuracy                           0.77      1180
   macro avg       0.81      0.62      0.67      1180
weighted avg       0.79      0.77      0.75      1180

[[717  19  12]
 [101  80   2]
 [134   2 113]]


In [25]:
try_clf_with_feature_selected(rf_clf,feature_names_prosody,y_train_e,y_test_e)
try_clf_with_feature_selected(rf_clf,selected_feature_names,y_train_e,y_test_e)

              precision    recall  f1-score   support

       Anger       0.70      0.80      0.75       187
    Calmness       0.47      0.88      0.61        16
     Disgust       0.53      0.43      0.47       187
        Fear       0.63      0.49      0.55       187
   Happiness       0.65      0.48      0.55       187
  Neutrality       0.56      0.65      0.60       167
     Sadness       0.54      0.74      0.63       187
    Surprise       0.89      0.77      0.83        62

    accuracy                           0.61      1180
   macro avg       0.62      0.65      0.62      1180
weighted avg       0.62      0.61      0.60      1180

[[149   0  13   8  14   2   1   0]
 [  0  14   0   0   0   0   2   0]
 [ 12   1  80  15  16  30  32   1]
 [ 14   0  12  92   9  15  44   1]
 [ 36   3  19  12  90  18   7   2]
 [  0   5  15   8   5 108  26   0]
 [  0   7   8   7   4  21 138   2]
 [  1   0   4   4   1   0   4  48]]
              precision    recall  f1-score   support

       Anger 

In [ ]:
start = time.time()
sfs = SequentialFeatureSelector(estimator=rf_est, 
                            n_features_to_select='auto',  # Select one feature at a time
                            direction='forward',  # Forward selection
                            scoring='balanced_accuracy',  # Use balabced accuracy as the evaluation metric - average of recall
                            cv=5)  # k-fold cross-validation
sfs.fit(X_train, y_train_s)

selected_feature_names = [feature_names_prosody[i] for i, selected in enumerate(sfs.get_support()) if selected]
print(f"{len(selected_feature_names)} of {len(feature_names_prosody)} selected. \nFeature names:", selected_feature_names)
X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)
rf_est.fit(X_train_selected, y_train_s)
accuracy = rf_est.score(X_test_selected, y_test_s)
print("Accuracy with selected features:", accuracy)
print(f"time taken: {round(time.time()-start)} seconds.")
print("selected: ", selected_feature_names)

17 of 34 selected.   
Feature names: ['Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum', 'Spectrum_band_density', 'Spectrum_band_energy', 'Formant_f1_mean', 'Formant_mff', 'HNR_stddev_hnr', 'HNR_mean_hnr', 'HNR_relative_min_hnr_time', 'Intensity_q3_intensity', 'Intensity_relative_max_intensity_time', 'Pitch_pitch_slope_without_octave_jumps', 'Pitch_stddev_pitch', 'Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Local Jitter']  
Accuracy with selected features: 0.7694915254237288  
time taken: 2287 seconds.  
selected:  ['Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum', 'Spectrum_band_density', 'Spectrum_band_energy', 'Formant_f1_mean', 'Formant_mff', 'HNR_stddev_hnr', 'HNR_mean_hnr', 'HNR_relative_min_hnr_time', 'Intensity_q3_intensity', 'Intensity_relative_max_intensity_time', 'Pitch_pitch_slope_without_octave_jumps', 'Pitch_stddev_pitch', 'Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Local Jitter']  

# End

### Feature Tables Part 2 

| feature category   | num of sub attributes  |
| ------------------ | ---------------------- |
|Intensity           |    9                   |
|pitch               |   12                   |
|HNR                 |    6                   |
|GNE                 |    5                   |
|Local Jitter        |    1                   |
|Local Shimmer       |    1                   |
|Spectrum            |    9                   | 
|formant             |   14                   |